# How to generate a metareport?

### Create a metareport comparing synthetic datasets with respect to a list of metrics. /!\ Only for the summary.

Assume that the synthetic datasets to compare are already generated \
Wisconsin Breast Cancer Dataset (WBCD)

In [1]:
# Standard library
import sys
import tempfile
from pathlib import Path

sys.path.append("..")

# 3rd party packages
import pandas as pd

# Local packages
import config
import clover.utils.draw
from clover.metrics.metareport import Metareport

## Load the real and synthetic Wisconsin Breast Cancer Datasets

In [2]:
df_real = {}
df_real["train"] = pd.read_csv("../data/WBCD_train.csv")
df_real["test"] = pd.read_csv("../data/WBCD_test.csv")
df_real["train"].shape

(455, 10)

### Choose the synthetic dataset

In [3]:
# generated by Synthpop here
df_synth = {
    "train": pd.read_csv(
        "../results/attack/data/1st_generation/2025-06-18_Synthpop_455samples.csv"
    ),
    "test": pd.read_csv(
        "../results/attack/data/1st_generation/2025-06-18_Synthpop_228samples.csv"
    ),
    "2nd_gen": pd.read_csv(
        "../results/attack/data/2nd_generation/2025-06-18_Synthpop_455samples.csv"
    ),
}

# random synthetic dataset to compare to the one generated by Synthpop
df_mock = {
    "train": df_synth["train"].apply(
        lambda x: x.sample(frac=1, replace=True).to_numpy()
    ),
    "test": df_synth["test"].apply(lambda x: x.sample(frac=1, replace=True).to_numpy()),
    "2nd_gen": df_synth["2nd_gen"].apply(
        lambda x: x.sample(frac=1, replace=True).to_numpy()
    ),
}

synth_datasets = {"synthpop": df_synth, "random": df_mock}

## Configure the metadata dictionary

### The continuous and categorical variables need to be specified, as well as the variable to predict

In [4]:
metadata = {
    "continuous": [
        "Clump_Thickness",
        "Uniformity_of_Cell_Size",
        "Uniformity_of_Cell_Shape",
        "Marginal_Adhesion",
        "Single_Epithelial_Cell_Size",
        "Bland_Chromatin",
        "Normal_Nucleoli",
        "Mitoses",
        "Bare_Nuclei",
    ],
    "categorical": ["Class"],
    "variable_to_predict": "Class",
}

## Generate the metareport

In [5]:
parameters = {  # see the notebooks utility_report and privacy_report for more details
    "cross_learning": False,
    "num_repeat": 1,
    "num_kfolds": 2,
    "num_optuna_trials": 15,
    "use_gpu": True,
    "sampling_frac": 0.5,
}

In [6]:
metareport = Metareport(
    dataset_name="Wisconsin Breast Cancer Dataset",
    df_real=df_real,
    synthetic_datasets=synth_datasets,
    metadata=metadata,
    figsize=(8, 6),  # will be automatically adjusted for larger or longer figures
    random_state=42,  # for reproducibility purposes
    metareport_folderpath=None,  # a dictionary containing the path of each already computed report to load and compare
    metrics=None,  # list of the metrics to compute. Can be utility or privacy metrics. If not specified, all the metrics are computed.
    params=parameters,  # the dictionary containing the parameters for both utility and privacy reports
)

In [7]:
metareport.compute()

/data8/install/anaconda3/envs/synthetic_data_p3.10/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:31:25] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


GAN-Leaks test set shape: (228, 10)
Monte Carlo Membership test set shape: (228, 10)
Detector test set shape: (228, 10)
LOGAN test set shape: (228, 10)
TableGan test set shape: (228, 10)
GAN-Leaks test set shape: (228, 10)


/data8/install/anaconda3/envs/synthetic_data_p3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Monte Carlo Membership test set shape: (228, 10)
Detector test set shape: (228, 10)
LOGAN test set shape: (228, 10)
TableGan test set shape: (228, 10)


## Get the summary report as a pandas dataframe

In [8]:
df_summary = metareport.summary()

In [9]:
df_summary

compared,random,synthpop
metric,,
cat_consis-within_ratio,1.000000,1.000000
cat_stats-frequency_coverage,0.975824,0.956044
cat_stats-support_coverage,1.000000,1.000000
classif-diff_real_synth,0.065878,0.004307
collision-avg_num_appearance_collision_real,6.428571,3.403509
collision-avg_num_appearance_collision_synth,1.285714,3.263158
collision-avg_num_appearance_realcontrol,1.349112,1.349112
collision-avg_num_appearance_realtrain,1.463023,1.463023
collision-avg_num_appearance_synth,1.006637,1.413043


### Style the result

The best value (minimum or maximal according to the submetric objective) is colored in green. The worst in yellow.

In [10]:
s = df_summary.style.pipe(Metareport.make_pretty, metrics=list(df_summary.index))
s

compared,random,synthpop
metric,,
cat_consis-within_ratio,1.00,1.00
cat_stats-frequency_coverage,0.98,0.96
cat_stats-support_coverage,1.00,1.00
classif-diff_real_synth,0.07,0.00
collision-avg_num_appearance_collision_real,6.43,3.40
collision-avg_num_appearance_collision_synth,1.29,3.26
collision-avg_num_appearance_realcontrol,1.35,1.35
collision-avg_num_appearance_realtrain,1.46,1.46
collision-avg_num_appearance_synth,1.01,1.41


### Save the styled result as html

In [11]:
with tempfile.TemporaryDirectory() as temp_dir:
    with open(Path(temp_dir) / "df.html", "w") as f:
        print(s.to_html(), file=f)

## Save and load the metareport

In [12]:
with tempfile.TemporaryDirectory() as temp_dir:
    metareport.save(savepath=temp_dir)  # save
    new_report = Metareport(
        metareport_folderpath={"synthpop": temp_dir, "random": temp_dir}
    )  # load